In [1]:
from datetime import datetime as dt
import math
from time import sleep
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
from ib_insync import *
import nest_asyncio
import random
import riskfolio as rf
import warnings
from waiting import wait, TimeoutExpired

In [2]:
nest_asyncio.apply()
ib = IB()
ib.connect('127.0.0.1', 7497, clientId = random.randint(1,99))

<IB connected to 127.0.0.1:7497 clientId=11>

In [3]:
#pd.options.display.float_format = "{:.4%}".format
warnings.filterwarnings("ignore")

In [4]:
#List of stocks currently listed on the Nasdaq
nasdaq_stocks = pd.read_html("http://en.wikipedia.org/wiki/Nasdaq-100#Components", match = 'Company')[0]
nasdaq_stocks.head()

,Company,Ticker,GICS Sector,GICS Sub-Industry
0,Adobe Inc.,ADBE,Information Technology,Application Software
1,Advanced Micro Devices,AMD,Information Technology,Semiconductors
2,Airbnb,ABNB,Consumer Discretionary,"Hotels, Resorts & Cruise Lines"
3,Alphabet Inc. (Class A),GOOGL,Communication Services,Interactive Media & Services
4,Alphabet Inc. (Class C),GOOG,Communication Services,Interactive Media & Services


In [5]:
nasdaq_ticker_list = []
nasdaq_ticker_list = nasdaq_stocks.Ticker.tolist()

In [6]:
nasdaq_ohlcv_data = {} # empty dictionary which will be filled with ohlcv dataframe for each ticker
for nasdaq_ticker in nasdaq_ticker_list: 
    nasdaq_ohlcv_data[nasdaq_ticker] = yf.download(nasdaq_ticker, period = 'max', interval = '1mo')
    nasdaq_ohlcv_data[nasdaq_ticker].dropna(inplace = True, how = 'all')
nasdaq_tickers = nasdaq_ohlcv_data.keys()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [7]:
# Calculating the performance KPIs

# Calculating the Compounded Annual Growth Rate
def CAGR(DF):
    df = DF.copy()
    #df['mon_return'] = df['Adj Close'].pct_change()
    df['cum_return'] = (1 + df['mon_return']).cumprod()
    n = len(df)/12 # the denominator is the number of trading periods in a year. In this case, 12 months
    CAGR = (df['cum_return'].tolist()[-1]) ** (1/n)-1
    return CAGR

# Calculating the Volatility
def volatility(DF):
    df = DF.copy()
    #df['return'] = df['Adj Close'].pct_change()
    vol  = df['mon_return'].std() * np.sqrt(12) # The number in the sqrt is the number of r trading periods in a year
    return vol

# Calculating the Sharpe Ratio
def sharpe(DF, rf):
    df = DF.copy()
    sharpe = (CAGR(df) - .03) / volatility(df)
    return sharpe

# Calulating the maximum drawdown
def max_dd(DF):
    df = DF.copy()
    #df['mon_return'] = df['Adj Close'].pct_change()
    df['cum_return'] = (1 + df['mon_return']).cumprod()
    df['cum_roll_max'] = df['cum_return'].cummax()
    df['drawdown'] = df['cum_roll_max'] - df['cum_return']
    df['drawdown_pct'] = df['drawdown'] / df['cum_roll_max']
    max_dd = df['drawdown_pct'].max()
    return max_dd

In [8]:
nasdaq_ohlc_dict = copy.deepcopy(nasdaq_ohlcv_data)
nasdaq_return_df = pd.DataFrame()
for nasdaq_ticker in nasdaq_tickers:
    print('calculating monthly return for ',nasdaq_ticker)
    nasdaq_ohlc_dict[nasdaq_ticker]['mon_return'] = nasdaq_ohlc_dict[nasdaq_ticker]['Close'].pct_change()
    nasdaq_return_df[nasdaq_ticker] = nasdaq_ohlc_dict[nasdaq_ticker]['mon_return']
nasdaq_return_df.dropna(how='all',axis=0, inplace=True)

calculating monthly return for  ADBE
calculating monthly return for  AMD
calculating monthly return for  ABNB
calculating monthly return for  GOOGL
calculating monthly return for  GOOG
calculating monthly return for  AMZN
calculating monthly return for  AEP
calculating monthly return for  AMGN
calculating monthly return for  ADI
calculating monthly return for  ANSS
calculating monthly return for  AAPL
calculating monthly return for  AMAT
calculating monthly return for  APP
calculating monthly return for  ARM
calculating monthly return for  ASML
calculating monthly return for  AZN
calculating monthly return for  TEAM
calculating monthly return for  ADSK
calculating monthly return for  ADP
calculating monthly return for  AXON
calculating monthly return for  BKR
calculating monthly return for  BIIB
calculating monthly return for  BKNG
calculating monthly return for  AVGO
calculating monthly return for  CDNS
calculating monthly return for  CDW
calculating monthly return for  CHTR
calculati

In [9]:
def pflio(DF,m,x):
    """Returns cumulative portfolio return
    DF = dataframe with monthly return info for all stocks
    m = number of stock in the portfolio
    x = number of underperforming stocks to be removed from portfolio monthly"""
    df = DF.copy()
    portfolio = []
    monthly_ret = [0]
    for i in range(len(df)):
        if len(portfolio) > 0:
            monthly_ret.append(df[portfolio].iloc[i,:].mean())
            bad_stocks = df[portfolio].iloc[i,:].sort_values(ascending=True)[:x].index.values.tolist()
            portfolio = [t for t in portfolio if t not in bad_stocks]
        fill = m - len(portfolio)
        new_picks = df.iloc[i,:].sort_values(ascending=False)[:fill].index.values.tolist()
        portfolio = portfolio + new_picks
        print('\n list of stocks to go long: \n', portfolio)
    return portfolio

In [10]:
nasdaq_long_list = pflio(nasdaq_return_df,10,6)


 list of stocks to go long: 
 ['ADSK', 'MU', 'INTC', 'AMD', 'AAPL', 'AMAT', 'ADP', 'LRCX', 'CSX', 'PAYX']

 list of stocks to go long: 
 ['ADSK', 'AAPL', 'CSX', 'PAYX', 'ADBE', 'PAYX', 'MNST', 'HON', 'MSFT', 'ADI']

 list of stocks to go long: 
 ['ADSK', 'CSX', 'ADBE', 'MSFT', 'MSFT', 'ADBE', 'AMGN', 'ADP', 'PCAR', 'ADSK']

 list of stocks to go long: 
 ['ADSK', 'MSFT', 'MSFT', 'ADSK', 'MSFT', 'AAPL', 'AMD', 'INTC', 'TXN', 'ADSK']

 list of stocks to go long: 
 ['AAPL', 'TXN', 'PAYX', 'ADBE', 'CTAS', 'AAPL', 'CSX', 'ADI', 'TXN', 'EXC']

 list of stocks to go long: 
 ['AAPL', 'ADBE', 'AAPL', 'ADI', 'KLAC', 'MSFT', 'INTC', 'LRCX', 'AMAT', 'AAPL']

 list of stocks to go long: 
 ['AAPL', 'ADBE', 'AAPL', 'AAPL', 'ROST', 'AMGN', 'ADSK', 'ADBE', 'AAPL', 'MNST']

 list of stocks to go long: 
 ['ADBE', 'AMGN', 'ADSK', 'ADBE', 'ADBE', 'ADSK', 'MSFT', 'MU', 'TXN', 'COST']

 list of stocks to go long: 
 ['ADBE', 'ADBE', 'ADBE', 'MU', 'ADBE', 'AMAT', 'MU', 'AAPL', 'INTC', 'LRCX']

 list of stocks 

In [11]:
sp_stocks = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
sp_stocks.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [12]:
sp_ticker_list = []
sp_ticker_list = sp_stocks.Symbol.tolist()

In [13]:
# Changing the ticker format to match the format in Yahoo finance for the following BRK.B and BF.B
sp_ticker_list[sp_ticker_list.index('BRK.B')] = 'BRK-B'
sp_ticker_list[sp_ticker_list.index('BF.B')] = 'BF-B'

In [14]:
sp_ohlcv_data = {} # empty dictionary which will be filled with ohlcv dataframe for each ticker
for sp_ticker in sp_ticker_list: 
    sp_ohlcv_data[sp_ticker] = yf.download(sp_ticker, period = 'max', interval = '1mo')
    sp_ohlcv_data[sp_ticker].dropna(inplace = True, how = 'all')
sp_tickers = sp_ohlcv_data.keys()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [15]:
sp_ohlc_dict = copy.deepcopy(sp_ohlcv_data)
sp_return_df = pd.DataFrame()
for sp_ticker in sp_tickers:
    print('calculating monthly return for ',sp_ticker)
    sp_ohlc_dict[sp_ticker]['mon_return'] = sp_ohlc_dict[sp_ticker]['Close'].pct_change()
    sp_return_df[sp_ticker] = sp_ohlc_dict[sp_ticker]['mon_return']
sp_return_df.dropna(how='all',axis=0, inplace=True)

calculating monthly return for  MMM
calculating monthly return for  AOS
calculating monthly return for  ABT
calculating monthly return for  ABBV
calculating monthly return for  ACN
calculating monthly return for  ADBE
calculating monthly return for  AMD
calculating monthly return for  AES
calculating monthly return for  AFL
calculating monthly return for  A
calculating monthly return for  APD
calculating monthly return for  ABNB
calculating monthly return for  AKAM
calculating monthly return for  ALB
calculating monthly return for  ARE
calculating monthly return for  ALGN
calculating monthly return for  ALLE
calculating monthly return for  LNT
calculating monthly return for  ALL
calculating monthly return for  GOOGL
calculating monthly return for  GOOG
calculating monthly return for  MO
calculating monthly return for  AMZN
calculating monthly return for  AMCR
calculating monthly return for  AEE
calculating monthly return for  AEP
calculating monthly return for  AXP
calculating monthly 

In [16]:
sp_long_list = pflio(sp_return_df,10,6)


 list of stocks to go long: 
 ['IP', 'MMM', 'GE', 'IBM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD']

 list of stocks to go long: 
 ['ABBV', 'ACN', 'ADBE', 'AMD', 'MMM', 'GE', 'IP', 'IBM', 'AOS', 'ABT']

 list of stocks to go long: 
 ['ADBE', 'AMD', 'AOS', 'ABT', 'MMM', 'GE', 'IP', 'IBM', 'AOS', 'ABT']

 list of stocks to go long: 
 ['AOS', 'ABT', 'AOS', 'ABT', 'GE', 'IP', 'MMM', 'IBM', 'AOS', 'ABT']

 list of stocks to go long: 
 ['GE', 'IP', 'IBM', 'MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD']

 list of stocks to go long: 
 ['ABBV', 'ACN', 'ADBE', 'AMD', 'IBM', 'GE', 'MMM', 'IP', 'AOS', 'ABT']

 list of stocks to go long: 
 ['ADBE', 'AMD', 'AOS', 'ABT', 'MMM', 'GE', 'IBM', 'IP', 'AOS', 'ABT']

 list of stocks to go long: 
 ['AOS', 'ABT', 'AOS', 'ABT', 'IP', 'GE', 'MMM', 'IBM', 'AOS', 'ABT']

 list of stocks to go long: 
 ['GE', 'IP', 'IBM', 'MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD']

 list of stocks to go long: 
 ['ABBV', 'ACN', 'ADBE', 'AMD', 'MMM', 'IBM', 'IP', 'GE', 

In [17]:
long_list = sp_long_list + nasdaq_long_list
long_list

['PM',
 'MOS',
 'HCA',
 'NEM',
 'SMCI',
 'VST',
 'GEV',
 'KDP',
 'RL',
 'AXON',
 'MDLZ',
 'MSTR',
 'MDLZ',
 'PDD',
 'TTD',
 'PDD',
 'MSTR',
 'KDP',
 'AXON',
 'CEG']

In [18]:
sleep(60)
port_returns = (
    yf.download(
        long_list,
        period  = '1mo' 
    )["Close"]
    .pct_change()
    .dropna()
)

[*********************100%***********************]  15 of 15 completed


In [19]:
factors = ["MTUM", "QUAL", "VLUE", "SIZE", "USMV"]

In [20]:
factor_returns = (
    yf.download(
        factors, 
        period  = '1mo'
    )["Close"]
    .pct_change()
    .dropna()
)

[*********************100%***********************]  5 of 5 completed


In [21]:
port = rf.Portfolio(returns = port_returns)

port.assets_stats(method_mu = "hist", 
                  method_cov = "ledoit"
                  )

port.lowerret = 0.00056488 * 1.5

loadings = rf.loadings_matrix(
    X = factor_returns,
    Y = port_returns, 
    feature_selection = "PCR",
    n_components = 0.95
)

In [22]:
loadings.style.format("{:.4f}").background_gradient(cmap='RdYlGn')

,const,MTUM,QUAL,SIZE,USMV,VLUE
AXON,0.0059,1.0899,0.8911,0.4030,-1.5115,-0.4213
CEG,-0.0005,1.2420,0.9874,0.4129,-1.8379,-0.5466
GEV,0.0056,1.3598,1.1116,0.5024,-1.8867,-0.5261
HCA,0.0049,-0.4921,-0.2156,0.1256,1.4493,0.6324
KDP,0.0019,0.0572,0.0849,0.0840,0.0773,0.0682
MDLZ,0.0011,-0.4094,-0.2348,0.0132,0.9779,0.3947
MOS,0.0077,-0.2008,-0.0455,0.1212,0.7658,0.3586
MSTR,0.0199,2.3563,1.9000,0.8276,-3.3765,-0.9735
NEM,0.0078,0.1097,0.1088,0.0720,-0.0736,0.0029
PDD,0.0063,0.3402,0.3619,0.2638,-0.1278,0.0669


In [23]:
port.factors = factor_returns

port.factors_stats(
    method_mu = "hist",
    method_cov = "ledoit",
)

In [24]:
w = port.optimization(
    model = "FM",
    rm = "MV",
    obj = "Sharpe",
    #rm = 'CVaR', 
    #obj = 'MaxRet',
    hist = True
)

In [25]:
w.reset_index(inplace = True)
w.rename(columns={'index':'ticker', 'weights':'weights'}, inplace = True)
w.sort_values('weights')

,ticker,weights
13,TTD,7.671800e-12
11,RL,7.992852e-12
1,CEG,9.492943e-12
2,GEV,1.401271e-11
9,PDD,1.875044e-11
7,MSTR,2.513236e-11
14,VST,3.375859e-11
10,PM,1.479961e-10
0,AXON,1.250792e-02
4,KDP,1.821086e-02


In [26]:
w[w['weights'] < .01] = np.nan
w.dropna(inplace = True)
w


,ticker,weights
0,AXON,0.012508
3,HCA,0.251758
4,KDP,0.018211
5,MDLZ,0.085180
6,MOS,0.250006
8,NEM,0.336263
12,SMCI,0.046074


In [27]:
w.sort_values('weights')

,ticker,weights
0,AXON,0.012508
4,KDP,0.018211
12,SMCI,0.046074
5,MDLZ,0.085180
6,MOS,0.250006
3,HCA,0.251758
8,NEM,0.336263


In [28]:
long_dict = {} # empty dictionary which will be filled with ohlcv dataframe for each ticker
op_ticker_list = w['ticker'].to_list()
for long_op_ticker in op_ticker_list:
    long_dict[long_op_ticker] = yf.download(long_op_ticker, period = '1mo')
    long_dict[long_op_ticker].dropna(inplace = True, how = 'all')
tickers = long_dict.keys()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [29]:
def buy_stock(long_ticker: str, buy_diff: float = None):
        
        # Checking the account balance
        acct_bal = float(ib.accountSummary()[9].value)
        
        stock = Stock(
            symbol = long_ticker, 
            exchange = 'SMART', 
            currency = 'USD'
        )
        
        purchase_amount = ((float(w[w['ticker'] == f'{long_ticker}']['weights']) * acct_bal)) / (long_dict[long_ticker]['Close'].iloc[-1])

        if buy_diff is None:
            action = Order(
                action = 'BUY', 
                totalQuantity = round(purchase_amount.item()), 
                orderType = 'MKT',  
                tif = 'GTC', 
                outsideRth = True
            )
        
        else:
            action = Order(
                action = 'BUY', 
                totalQuantity = math.ceil(buy_diff), 
                orderType = 'MKT',  
                tif = 'GTC', 
                outsideRth = True
            )  
        
        order = ib.placeOrder(stock, action)

In [30]:
def sell_stock(ticker: str, sell_diff: float = None):
        stock = Stock(
            symbol = ticker, 
            exchange = 'SMART', 
            currency = 'USD'
        )
        
        for i in range (len(ib.positions())):
            if ib.positions()[i].contract.symbol == ticker:
                sell_amount = ib.positions()[i].position

        if sell_diff is None:
            action = Order(
                action = 'SELL', 
                totalQuantity = sell_amount, 
                orderType = 'MKT',  
                tif = 'GTC', 
                outsideRth = True
            )
        else:
            action = Order(
                action = 'SELL', 
                totalQuantity = math.ceil(sell_diff), 
                orderType = 'MKT',  
                tif = 'GTC', 
                outsideRth = True
            ) 
        
        order = ib.placeOrder(stock, action)

In [31]:
current_holdings = []

for i in range(len(ib.positions())):
    current_holdings.append(ib.positions()[i].contract.symbol)

In [32]:
for holdings in current_holdings:
    if holdings in long_list:
        for i in range(len(ib.positions())):
            if ib.positions()[i].contract.symbol == holdings:
                current_holding_port_percentage = ib.positions()[i].position * ib.positions()[i].avgCost / float(ib.accountSummary()[24].value)
                if not w[w['ticker'] == f'{holdings}'].empty:
                    new_holding_port_percentage = w[w['ticker'] == f'{holdings}']['weights'].iloc[0]
                    print(f'\n{holdings} current percentage of the portfolio {current_holding_port_percentage} % new percentage {new_holding_port_percentage} %')
                    
                    if current_holding_port_percentage > new_holding_port_percentage:
                        sell_diff = current_holding_port_percentage - new_holding_port_percentage
                        print(f'The sell diff for {holdings} is {sell_diff} %')
                    
                    elif current_holding_port_percentage < new_holding_port_percentage:
                        buy_diff = new_holding_port_percentage - current_holding_port_percentage
                        print(f'The buy diff for {holdings} is {buy_diff} %')
                else:
                    print(f'{holdings} not found in the weights DataFrame')


PM not found in the weights DataFrame


In [33]:
current_holdings = []

for i in range(len(ib.positions())):
    current_holdings.append(ib.positions()[i].contract.symbol)

for holdings in current_holdings:
    if holdings not in long_list: 
        try:
            sell_stock(holdings)
        except TimeoutExpired as timeout:
            print(f'Sale of {holdings} {timeout} ')
    
    elif holdings in long_list:
        for i in range(len(ib.positions())):
            if ib.positions()[i].contract.symbol == holdings:
                current_holding_port_percentage = ib.positions()[i].position * ib.positions()[i].avgCost / float(ib.accountSummary()[24].value)
                if not w[w['ticker'] == f'{holdings}'].empty:
                    new_holding_port_percentage = w[w['ticker'] == f'{holdings}']['weights'].iloc[0]
                    
                    if current_holding_port_percentage > new_holding_port_percentage:
                        sell_diff = current_holding_port_percentage - new_holding_port_percentage
                        try:
                            sell_stock(ticker = holdings, sell_diff = sell_diff)
                        except TimeoutExpired as timeout:
                            print(f'Sale of {holdings} has been after {timeout}')
                    
                    elif current_holding_port_percentage < new_holding_port_percentage:
                        buy_diff = new_holding_port_percentage - current_holding_port_percentage
                        try:
                            buy_stock(long_ticker = holdings, buy_diff = buy_diff)
                        except ValueError as e:
                            print(f'Error buying {holdings}: {e}')
                else:
                    sell_stock(holdings)
                    print(f'{holdings} not found in the weights DataFrame, Therefore {holdings} was sold')
    

PM not found in the weights DataFrame, Therefore PM was sold


In [34]:
for long_op_ticker in op_ticker_list:
    if long_op_ticker not in current_holdings:
        try:
            buy_stock(long_op_ticker)
            ib.sleep(15)
        except ValueError as e:
            print(f'Error buying stock {long_op_ticker}: {e}')

In [35]:
# Get current positions
positions = ib.positions()

# Print the positions
for position in positions:
    print(f"Symbol: {position.contract.symbol}, Quantity: {position.position}")


Symbol: AXON, Quantity: 21.0
Symbol: HCA, Quantity: 724.0
Symbol: KDP, Quantity: 479.0
Symbol: MDLZ, Quantity: 1140.0
Symbol: MOS, Quantity: 8214.0
Symbol: NEM, Quantity: 5837.0
Symbol: SMCI, Quantity: 998.0


In [36]:
# Loop through your positions and creating trailing stop orders
for position in positions:
    contract = position.contract
    quantity = position.position
    #print(f'{contract} and the amount is {quantity}')
    
    if quantity > 0:  # Only create trailing stop orders for long positions
        # Define the trailing stop order
        trailing_stop_order = Order(
            action = 'SELL',  # Sell to exit the position
            orderType = 'TRAIL',
            totalQuantity = abs(quantity),
            trailingPercent = 10.0,  # Adjust the trailing percentage as needed
            tif = 'GTC'
        )

        # Submit the order
        trade = ib.placeOrder(contract, trailing_stop_order)
        print(f"Trailing stop order placed for {contract.symbol}, to sell a Quantity of {quantity}")

Trailing stop order placed for AXON, to sell a Quantity of 21.0
Trailing stop order placed for HCA, to sell a Quantity of 724.0
Trailing stop order placed for KDP, to sell a Quantity of 479.0
Trailing stop order placed for MDLZ, to sell a Quantity of 1140.0
Trailing stop order placed for MOS, to sell a Quantity of 8214.0
Trailing stop order placed for NEM, to sell a Quantity of 5837.0
Trailing stop order placed for SMCI, to sell a Quantity of 998.0


In [37]:
# Fetch open orders
open_orders = ib.openOrders()
for open_order in open_orders:
    print(f"Open Orders: {open_orders}")

ib.disconnect()

Open Orders: [Order(orderId=14, clientId=11, action='SELL', totalQuantity=21.0, orderType='TRAIL', tif='GTC', trailingPercent=10.0), Order(orderId=15, clientId=11, action='SELL', totalQuantity=724.0, orderType='TRAIL', tif='GTC', trailingPercent=10.0), Order(orderId=16, clientId=11, action='SELL', totalQuantity=479.0, orderType='TRAIL', tif='GTC', trailingPercent=10.0), Order(orderId=17, clientId=11, action='SELL', totalQuantity=1140.0, orderType='TRAIL', tif='GTC', trailingPercent=10.0), Order(orderId=18, clientId=11, action='SELL', totalQuantity=8214.0, orderType='TRAIL', tif='GTC', trailingPercent=10.0), Order(orderId=19, clientId=11, action='SELL', totalQuantity=5837.0, orderType='TRAIL', tif='GTC', trailingPercent=10.0), Order(orderId=20, clientId=11, action='SELL', totalQuantity=998.0, orderType='TRAIL', tif='GTC', trailingPercent=10.0)]
Open Orders: [Order(orderId=14, clientId=11, action='SELL', totalQuantity=21.0, orderType='TRAIL', tif='GTC', trailingPercent=10.0), Order(order